In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import hail as hl
import os.path as osp
import multiprocessing

# Benchmarking utilities
%run ../init/benchmark.py
register_timeop_magic(get_ipython(), 'hail')
data_dir = osp.expanduser('~/data/gwas/tutorial/2_PS_GWAS')

# from pyspark.sql import SparkSession
# spark = SparkSession.builder \
#        .master("local[*]") \
#        .appName("Hail") \
#        .config("spark.sql.shuffle.partitions", 1) \
#        .getOrCreate()
hl.init() 

Running on Apache Spark version 2.4.4
SparkUI available at http://3d498b83ee57:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.30-2ae07d872f43
LOGGING: writing to /home/eczech/repos/gwas-analysis/notebooks/tutorial/02-population-stratification/hail-20200116-1515-0.2.30-2ae07d872f43.log


In [5]:
# hl.spark_context()._conf.set("spark.sql.shuffle.partitions", 1)

In [2]:
# # Load PLINK dataset using our custom reference
# PS1_FILE = "ALL.2of4intersection.20100804.genotypes_no_missing_IDs"
# mt = hl.import_plink(
#     osp.join(data_dir, PS1_FILE + '.bed'),
#     osp.join(data_dir, PS1_FILE + '.bim'),
#     osp.join(data_dir, PS1_FILE + '.fam'),
#     skip_invalid_loci=True,
#     #reference_genome='hapmap3_hg18'
# )

2020-01-16 13:33:15 Hail: INFO: Found 629 samples in fam file.
2020-01-16 13:33:15 Hail: INFO: Found 25488488 variants in bim file.


In [2]:
# Load PLINK dataset using our custom reference
PS1_FILE = "ALL.2of4intersection.20100804.genotypes_no_missing_IDs"
#PS1_FILE = "ALL.2of4intersection.20100804.genotypes"
mt = hl.import_plink(
    osp.join(data_dir, PS1_FILE + '.bed'),
    osp.join(data_dir, PS1_FILE + '.bim'),
    osp.join(data_dir, PS1_FILE + '.fam'),
    skip_invalid_loci=False,
    reference_genome='GRCh37'
)

2020-01-16 15:18:14 Hail: INFO: Found 629 samples in fam file.
2020-01-16 15:18:14 Hail: INFO: Found 25488488 variants in bim file.


In [3]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'fam_id': str
    'pat_id': str
    'mat_id': str
    'is_female': bool
    'is_case': bool
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'cm_position': float64
----------------------------------------
Entry fields:
    'GT': call
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [ ]:
# mt.write(osp.join(data_dir, 'ALL.2of4intersection.20100804.genotypes_no_missing_IDs.mt'), overwrite=True)

In [3]:
mt.count()

2020-01-16 13:36:07 Hail: INFO: Found 629 samples in fam file.
2020-01-16 13:36:07 Hail: INFO: Found 25488488 variants in bim file.
2020-01-16 13:39:43 Hail: INFO: Coerced sorted dataset


(25488488, 629)

In [7]:
rscts = (
    mt.group_rows_by(mt.rsid)
    .aggregate(n=hl.agg.count())
)

In [8]:
rscts.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'fam_id': str
    'pat_id': str
    'mat_id': str
    'is_female': bool
    'is_case': bool
----------------------------------------
Row fields:
    'rsid': str
----------------------------------------
Entry fields:
    'n': int64
----------------------------------------
Column key: ['s']
Row key: ['rsid']
----------------------------------------


In [9]:
rscte = rscts.entries()

2020-01-16 15:19:53 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'


In [ ]:
rscte.group_by(rscte.n).aggregate(n_n=hl.agg.count()).show()

2020-01-16 15:22:41 Hail: INFO: Coerced sorted dataset


In [ ]:
cts = pd.Series(rscts)

In [ ]:
cts.head()

In [ ]:
# no missing ids
cts.value_counts()

In [7]:
# w/ missing ids
cts.value_counts()

1           15112987
10375501           1
dtype: int64

In [8]:
cts[cts > 1].head()

.    10375501
dtype: int64

In [ ]:
cts.value_counts()

In [ ]:
ss.read.format("plink").load(data_dir / "ALL.2of4intersection.20100804.genotypes_no_missing_IDs.bed" toString).count